# core

> Polars API logic is included in `core` for commonly reused data preprocess and transformations from bulk datasets obtained using Cryo

In [ ]:
#| default_exp core

In [ ]:
# | export
import jupyter_black
import polars as pl


pl.Config.set_fmt_str_lengths(200)
pl.Config.set_fmt_float("full")


jupyter_black.load()

In [ ]:
# | export
def get_txs_blocks_mempool(
    txs: pl.LazyFrame, blocks: pl.LazyFrame, mempool: pl.LazyFrame | None = None
) -> pl.LazyFrame:
    """
    Combines transaction, block, and optionally flashbots mempool data into a single LazyFrame.

    Uses pattern matching to handle the optional mempool data.
    If mempool data is provided, it is joined with the transactions and blocks data.
    If not, only transactions and blocks are joined.

    Parameters:
    - txs (pl.LazyFrame): LazyFrame containing transaction data.
    - blocks (pl.LazyFrame): LazyFrame containing block data.
    - address (str): Address used to filter the data (not currently used).
    - mempool (pl.LazyFrame, optional): LazyFrame containing mempool data. Default is None.

    Returns:
    - pl.LazyFrame: A unified LazyFrame with enriched transaction data.
    """
    # Use pattern matching to handle the presence or absence of mempool data
    match mempool:
        case _ if isinstance(mempool, pl.LazyFrame):
            # Join transactions with blocks and mempool data if mempool is provided
            combined_df = txs.join(
                blocks, on="block_number", how="left", suffix="_block"
            ).join(
                mempool.select("hash", "timestamp"),
                right_on="hash",
                left_on="transaction_hash",
                how="left",
                suffix="_mempool",
            )

        case None:
            print('None Type Expected')
            # Join only transactions with blocks if mempool is not provided
            combined_df = txs.join(
                blocks, on="block_number", how="left", suffix="_block"
            )

    # Common processing steps for both scenarios
    return (
        combined_df.with_columns(
            [
                # Calculate the transaction gas cost
                (pl.col("gas_used") * pl.col("gas_price") / 10**18).alias(
                    "tx_gas_cost"
                ),
                # Convert epoch timestamp to datetime
                pl.from_epoch("timestamp").alias("block_datetime"),
                # Calculate the max transaction index per block
                pl.col("transaction_index")
                .max()
                .over(pl.col("block_number"))
                .name.suffix("_max"),
                # Calculate the gas price premium over the base fee per gas
                (pl.col("gas_price") / pl.col("base_fee_per_gas")).alias(
                    "block_gas_premium"
                ),
            ]
        )
        .with_columns(
            # Calculate the transaction index percentile within its block
            (pl.col("transaction_index") / pl.col("transaction_index_max") * 100).alias(
                "blockspace_percentile"
            )
        )
        .with_columns(
            # Round the block space percentile for easier interpretation
            (pl.col("blockspace_percentile").round()).alias(
                "rounded_blockspace_percentile"
            )
        )
        .fill_nan(0)  # Fill NaN values with 0
        .unique()  # Ensure all rows are unique
    )


# | export
def filter_from_address(df: pl.LazyFrame, address: str) -> pl.LazyFrame:
    """
    Filters a LazyFrame by the 'from_address' column.

    This function compares the 'from_address' column in the LazyFrame with the provided address.
    It returns a new LazyFrame containing only the rows where the 'from_address' matches the given address.

    Parameters:
    - df (pl.LazyFrame): The LazyFrame to filter.
    - address (str): The address to filter by.

    Returns:
    - pl.LazyFrame: Filtered LazyFrame.
    """
    # Convert the address to lowercase for case-insensitive comparison
    return df.filter(pl.col("from_address") == address.lower())


# | export
def filter_to_address(df: pl.LazyFrame, address: str) -> pl.LazyFrame:
    """
    Filters a LazyFrame by the 'to_address' column.

    This function compares the 'to_address' column in the LazyFrame with the provided address.
    It returns a new LazyFrame containing only the rows where the 'to_address' matches the given address.

    Parameters:
    - df (pl.LazyFrame): The LazyFrame to filter.
    - address (str): The address to filter by.

    Returns:
    - pl.LazyFrame: Filtered LazyFrame.
    """
    # Convert the address to lowercase for case-insensitive comparison
    return df.filter(pl.col("to_address") == address.lower())

For example, this will talk briefly about how to use the above functions.